<a href="https://colab.research.google.com/github/leonaardoas/Demand-Forecast-/blob/main/Previs%C3%A3o_de_Demanda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error as mae

In [41]:
read = pd.read_csv('/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados/relatorio_01_2020.csv', sep=";")
read

,Produto,,Código,Localização,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda
0,Camisa Manga Curta - Castelo Novo Tamanho:6 a...,,MC-CASTELO6,Estoque,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00"
1,Camisa Manga Curta - Castelo Novo Tamanho:10 ...,,MC-CASTELO10,Estoque,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50"
2,Camisa Manga Curta - Castelo Novo Tamanho:8 a...,,MC-CASTELO8,Estoque,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00"
3,Camisa Manga Curta - Castelo Novo Tamanho:12 ...,,MC-CASTELO12,Estoque,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25"
4,Camisa Manga Curta - Castelo Novo Tamanho:14 ...,,MC-CASTELO14,Estoque,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Jaqueta Fundamental - Maple Bear Tamanho:16 an...,,JMICF-16,NaN,"1,00","0,00","123,80","123,80","0,00","0,00","0,00","0,00","123,80"
358,Camisa Pólo - Colégio Bruno Ostmann Tamanho:P,,pol-cbop,Estoque,"1,00","0,00","55,00","55,00","0,00","0,00","0,00","0,00","55,00"
359,Short Saia Microfibra 100% Poliéster Castelo E...,,ss-castinf4,NaN,"1,00","1,00","40,95","40,95","40,95","0,00","0,00","0,00","40,95"
360,Camiseta Regata Branca - Colégio Aprovado Tama...,,regb-apv16,Estoque,"1,00","1,00","47,00","47,00","47,00","0,00","0,00","0,00","47,00"


In [42]:

def carregar_dados(arquivo):
  data = pd.read_csv(f"/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados/{arquivo}", sep=";")
  filename = arquivo.replace(".csv", "")
  filename = filename.split("_")
  month = filename[1]
  year = filename[2]
  data.insert(0, "day", 1) #posição, nome da coluna, elemento a ser inserido
  data.insert(0, "month", month)
  data.insert(0, "year", year)
  data["date"] = pd.to_datetime(data[["year", "month", "day"]])
  data.drop(["day", "month", "year"], axis=1, inplace=True)

  return data

In [43]:
sheets = []

for arquivo in os.listdir("/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados"):
  func = carregar_dados(arquivo)
  sheets.append(func)

dt = pd.concat(sheets, ignore_index=True)
dt


,Produto,,Código,Localização,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,Camisa Manga Curta - Castelo Novo Tamanho:6 a...,,MC-CASTELO6,Estoque,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,Camisa Manga Curta - Castelo Novo Tamanho:10 ...,,MC-CASTELO10,Estoque,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,Camisa Manga Curta - Castelo Novo Tamanho:8 a...,,MC-CASTELO8,Estoque,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,Camisa Manga Curta - Castelo Novo Tamanho:12 ...,,MC-CASTELO12,Estoque,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,Camisa Manga Curta - Castelo Novo Tamanho:14 ...,,MC-CASTELO14,Estoque,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4152,Camiseta Regata Infantil - CE Carvalho Tamanh...,,regi-car6,Estoque,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4153,Camiseta Regata Marinho - Colégio Aprovado Tam...,,regm-apv8,Estoque,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4154,Jaqueta Infantil - Maple Bear Tamanho:4 anos,,JMICI-4,NaN,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01
4155,Short Saia - CE Carvalho Tamanho:4 anos,,ssh-car4,NaN,"1,00","1,00","54,88","54,88","54,88","0,00","0,00","0,00","54,88",2023-04-01


In [44]:
totais = dt[dt.Produto == 'Totais']
totais = len(totais)
totais

35

In [46]:
df = dt[dt.Produto != 'Totais'].reset_index(drop=True)
df

,Produto,,Código,Localização,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,Camisa Manga Curta - Castelo Novo Tamanho:6 a...,,MC-CASTELO6,Estoque,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,Camisa Manga Curta - Castelo Novo Tamanho:10 ...,,MC-CASTELO10,Estoque,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,Camisa Manga Curta - Castelo Novo Tamanho:8 a...,,MC-CASTELO8,Estoque,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,Camisa Manga Curta - Castelo Novo Tamanho:12 ...,,MC-CASTELO12,Estoque,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,Camisa Manga Curta - Castelo Novo Tamanho:14 ...,,MC-CASTELO14,Estoque,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4117,Camiseta Regata Amarela - Colégio Ativo Tamanh...,,reg-am-ativo4,Estoque,"1,00","1,00","47,04","47,04","47,04","0,00","0,00","0,00","47,04",2023-04-01
4118,Camiseta Regata Infantil - CE Carvalho Tamanh...,,regi-car6,Estoque,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4119,Camiseta Regata Marinho - Colégio Aprovado Tam...,,regm-apv8,Estoque,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4120,Jaqueta Infantil - Maple Bear Tamanho:4 anos,,JMICI-4,NaN,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01
